In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline

In [10]:
from lightgbm import LGBMClassifier

In [11]:
import seaborn as sns
sns.set()

In [12]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
from sklearn.preprocessing import LabelEncoder
# train data 입력시 id, target 모두 넣어주어야 한다. 
# test data 입력시 id 모두 넣어주어야 한다.
# from sklearn.preprocessing import LabelEncoder 해주어야 함
def submit_kaggle(df_train, df_test, model, path, user_id = "id", target = "country_destination"):
    le = LabelEncoder()

    # feature, target 전처리
    y_train = le.fit_transform(df_train[target])
    X_train = df_train.drop([target, user_id], axis = 1)

    X_test_id = df_test[user_id] # id 따로 정의
    X_test = df_test.drop([user_id], axis = 1)

    # train data set으로 fitting
    print("model fitting 시작 !!")
    model = model.fit(X_train, y_train)
    predic_proba = model.predict_proba(X_test)
    print("model fitting 완료 !!")
    
    # submit data, predict data 생성
    df_submit = pd.DataFrame(columns=["id", "country"])

    ids = []
    cts = []
    for i in range(len(X_test_id)):
        idx = X_test_id.iloc[i]
        ids += [idx] * 5
        cts += le.inverse_transform(np.argsort(predic_proba[i])[::-1])[:5].tolist()

    df_submit["id"] = ids
    df_submit["country"] = cts
    
    df_submit.to_csv(path, index = False)
    print("csv file 제작 완료 !!")
    
    !kaggle competitions submit -c airbnb-recruiting-new-user-bookings -f {path} -m "Message"

### Load_data

In [14]:
df_train = pd.read_csv("../data/train_age.csv")
df_test = pd.read_csv("../data/test_age.csv")

In [15]:
df_session = pd.read_csv("../data/sessions.csv")

In [16]:
lgb = LGBMClassifier(max_depth=4 , n_jobs=4)

### action - count - action_all

0.87585

In [17]:
df_session_action_count = df_session.groupby(["user_id", "action"])["secs_elapsed"].count().fillna(0)

In [18]:
df_session_action_count = df_session_action_count.unstack().fillna(0)

In [19]:
df_train_new = df_train.merge(df_session_action_count, how = "left", left_on = "id", right_index = True).fillna(0)
df_test_new = df_test.merge(df_session_action_count, how = "left", left_on = "id", right_index = True).fillna(0)

In [29]:
submit_kaggle(df_train_new, df_test_new, lgb, "submit/submit.csv")

model fitting 시작 !!
model fitting 완료 !!
csv file 제작 완료 !!
Successfully submitted to Airbnb New User Bookings

### action_all + action_type_all - count

0.87659

In [20]:
df_session_action_type_count = df_session.groupby(["user_id", "action_type"])["secs_elapsed"].count().fillna(0)

In [21]:
df_session_action_type_count = df_session_action_type_count.unstack().fillna(0)

In [22]:
df_train_new = df_train_new.merge(df_session_action_type_count, how = "left", left_on = "id", right_index = True).fillna(0)
df_test_new = df_test_new.merge(df_session_action_type_count, how = "left", left_on = "id", right_index = True).fillna(0)

In [40]:
df_train_new.shape, df_test_new.shape

((213451, 517), (62096, 516))

In [ ]:
# make csv file

In [27]:
df_train_new.to_csv("../data/train_age_action_count.csv", index = False)
df_test_new.to_csv("../data/test_age_action_count.csv", index = False)

In [41]:
submit_kaggle(df_train_new, df_test_new, lgb, "submit/submit.csv")

model fitting 시작 !!
model fitting 완료 !!
csv file 제작 완료 !!
Successfully submitted to Airbnb New User Bookings